In [3]:
import json
import urllib.request
import json
from nltk import word_tokenize
import re
import collections
import pandas as pd

In [4]:
with open("transcripts/" + 'mini_transcript.txt' ,encoding='utf-8') as f:
    mini_script = f.read()


print(mini_script)

Sam: Here. 
Frodo gardner: What about you? 
Sam: Oh, no. I’m not hungry. Leastways not for lembas bread. 
Frodo: Sam… 
Sam: All right. We don’t have that much left. We have to be careful or we’re going to run out. You go ahead and eat that, Mister Frodo. I’ve rationed it. There should be enough. 
Frodo: For what? 
Sam: The journey home. 
 
Gollum: Come, Hobbitses. Very close now. Very close to Mordor. No safe places here. Hurry! 
 
Merry: It’s good. Definitely from the Shire. Longbottom Leaf.

Pippin: I feel like I’m back at the Green Dragon.
Merry: Green Dragon.
Pippin: A mug of ale in my hand. Putting my feet up on a settle after a hard day’s work.
Merry: Only, you’ve never done a hard day’s work. 



In [5]:
# CharDialogue = collections.defaultdict(list)

# #Character names from wiki
# charactersDf = pd.read_csv('characters.csv')

# characters = charactersDf.Name

# for character in characters:
#     #charName = mergeNames(splitLine[0].lower())
#     CharDialogue[character.lower()].append("")
    

# print(CharDialogue)


In [6]:


with open("transcripts/" + 'Fotr_parsed_cleaned2.txt' ,encoding='utf-8') as f:
    fellowshipScript = f.read()

with open("transcripts/" + '2t_parsed_and_cleaned.txt' ,encoding='utf-8') as f:
    towersScript = f.read()

with open("transcripts/" + 'RotK_parsed_and_cleaned.txt' ,encoding='utf-8') as f:
    kingScript = f.read()



In [7]:
# #þarf að splitta keyps in char dialogue en ekki charName


# def addToCharDict(charName, dialogue):
#     #charNameSplit = charName.split()
#     for n in charName:
#         #print(n)
#         if n in CharDialogue:
#             #print("appending", n)
#             print(n, " is found!")
#             CharDialogue[charName].append(dialogue)
#         else:
#             print(n, " is NOT found!")
#     #print(CharDialogue)

# #print(CharDialogue)
        


In [8]:
#Here be dragons...

# def mergeNames(charName):
#     splitName = charName.split()

#     for n in splitName:
#         if(n == "hobbits"):
#             return "hobbit"

#         else:
#             return n


In [9]:
#dict with name of all characters from wiki as key and all parts of name in a list as value 
#'Aragorn II Elessar': ['Aragorn', 'II', 'Elessar']

# wikiCharListNames = collections.defaultdict(list)

# for character in characters:
#     splitCharName = character.split()
#     for name in splitCharName:
#         wikiCharListNames[character].append(name)


#print(wikiCharListNames)

In [19]:
#characterDialogue = collections.defaultdict(list)
#fix frodo gardener
CharDialogue = collections.defaultdict(list)

def newEmptyCharDict():
    #þarf að splitta keyps in char dialogue en ekki charName
    

    #Character names from wiki
    charactersDf = pd.read_csv('characters.csv')

    characters = charactersDf.Name
    #print(characters)
    for character in characters:
        #charName = mergeNames(splitLine[0].lower())
        if character == "Frodo Gardner":
            #print("found gardener!")
            CharDialogue['gardner'].append("")
        else:
            CharDialogue[character.lower()].append("")
    CharDialogue["hobbits"].append("")
    CharDialogue["ring"].append("")
    CharDialogue["orcs"].append("")
    CharDialogue["men"].append("")
    #print(CharDialogue)


def addToCharDict(charName, dialogue):

    #translate bug
    substr = "Do I not also have your love"
    if dialogue.find(substr) != -1:
        return

    charNameSplit = charName.split()
    addSuccessful = False

    for wikiChar in CharDialogue:
        for name in charNameSplit:
            #a from a hobbit, and, of
            if len(name) < 3 or name == "and":
                #print("name", name)
                return
            if name in wikiChar:
                CharDialogue[wikiChar].append(dialogue)
                addSuccessful = True
            if name == "merry" and "meriadoc brandybuck" == wikiChar:
                CharDialogue["meriadoc brandybuck"].append(dialogue)
                addSuccessful = True
            if name == "pippin" and "peregrin took" == wikiChar:
                CharDialogue["peregrin took"].append(dialogue)
                addSuccessful = True
            if name == "strider" and "aragorn ii elessar" == wikiChar:
                CharDialogue["aragorn ii elessar"].append(dialogue)
                addSuccessful = True
            if name == "galadriel’s" and "galadriel" == wikiChar:
                CharDialogue["galadriel"].append(dialogue)
                addSuccessful = True
            if (name == "sméagol" or name == "smèagol") and "gollum" == wikiChar:
                 CharDialogue["gollum"].append(dialogue)
                 addSuccessful = True
            if name == "grima" and "gríma wormtongue" == wikiChar:
                CharDialogue["gríma wormtongue"].append(dialogue)
                addSuccessful = True
            if name == "èomer" and "éomer" == wikiChar:
                CharDialogue["éomer"].append(dialogue)
                addSuccessful = True
            if name == "èowyn" and "éowyn" == wikiChar:
                CharDialogue["éowyn"].append(dialogue)
                addSuccessful = True
            if name == "freya" and "freda" == wikiChar:
                CharDialogue["freda"].append(dialogue)
                addSuccessful = True
            if name == "hama" and "háma" == wikiChar:
                CharDialogue["háma"].append(dialogue)
                addSuccessful = True
            if (name == "soldier" or name == "captain" or name == "mother" or name == "crowd" or name == "gatekeeper" or name == "guard" or name == "refugees" or name == "soldiers") and "men" == wikiChar:
                CharDialogue["men"].append(dialogue)
                addSuccessful = True
            if name == "maúhur" and "mauhúr" == wikiChar:
                CharDialogue["mauhúr"].append(dialogue)
                addSuccessful = True
            if name == "eothain" and "éothain (film character)" == wikiChar:
                CharDialogue["éothain (film character)"].append(dialogue)
                addSuccessful = True
            if name == "eomer" and "éomer" == wikiChar:
                CharDialogue["éomer"].append(dialogue)
                addSuccessful = True
            if name == "theoden" and "théoden" == wikiChar:
                CharDialogue["théoden"].append(dialogue)
                addSuccessful = True

    return addSuccessful
            


def makeCharacterDialogue(scripts):
    #print(script.splitlines())
    allNotFound = []

    for script in scripts:
        lines = script.splitlines()

        for line in lines:
            #print(line)
            splitLine = line.split(":", 1)
            #print(splitLine)

            if len(splitLine) > 1:
                #merge together edge cases e.g. Hobbit / Hobbits
                #name = splitLine[0].lower()
                dialogue = splitLine[1]
                #charName = mergeNames(splitLine[0].lower())
                charName = splitLine[0].lower()
                addChar = addToCharDict(charName, dialogue)
                if addChar == False:
                    allNotFound.append(charName)
                #characterDialogue[splitLine[0]].append(dialog)

    print(allNotFound)

scripts = [fellowshipScript, towersScript, kingScript]
#scripts = [mini_script]
#scripts = [fellowshipScript]
newEmptyCharDict()
makeCharacterDialogue(scripts)


with open("characterDialogue.json", "wt", encoding='utf-8') as outfile:
    json.dump(CharDialogue, outfile, ensure_ascii=False)

['text', 'text', 'ringwraith', 'all', 'nazgûl', 'someone whispers', 'baglor', 'baglor', 'roc', 'roc', 'ringwraith', 'wizard', 'wizard', 'parn', 'parn', 'parn', 'lorne', 'malamir', 'malamir', 'hasslebak', '[translation', 'parn', 'parn', 'ringwraith', 'manathor', 'rohirrim', 'rohirrim', 'sauron’s voice']


we added hobbits and orcs to collect scripts without names

added ring

what happens to men?

fer orc bara á réttan stað?
A í a hobbit er að lenda alls staðar
[Translation: "Do I not also have your love?" 

hello a im, ú-'erin veleth lîn? [translation þarf að taka út

In [11]:
#scripts = [fellowshipScriptSplit, towersScriptSplit, kingScriptSplit]

# scripts = [mini_script]

# for script in scripts:
#     makeCharacterDialogue(script)



    


In [12]:
# characterDialogue = collections.defaultdict(list)

# def makeCharacterDialogue(scripts):
#     #print(script.splitlines())


#     for script in scripts:
#         lines = script.splitlines()

#         for line in lines:
#             #print(line)
#             splitLine = line.split(":", 1)
#             #print(splitLine)

#             if len(splitLine) > 1:
#                 #merge together edge cases e.g. Hobbit / Hobbits
#                 #name = splitLine[0].lower()
#                 dialog = splitLine[1]
#                 charName = mergeNames(splitLine[0].lower())
#                 characterDialogue[splitLine[0]].append(dialog)

#     #print(characterDialogue)

# scripts = [fellowshipScript, towersScript, kingScript]

# makeCharacterDialogue(scripts)

# with open("characterDialogue.json", "wt", encoding='utf-8') as outfile:
#     json.dump(characterDialogue, outfile, ensure_ascii=False)